In [308]:
from numpy import array ,eye, argmax
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM 

 
# 1. 데이터
input_w=4
input_string="떳다떳다비행기날아라날아라높이높이날아라우리비행기내가만든비행기날아라날아라높이높이날아라우리비행기"
char_set=sorted(set(input_string))
dict_size=len(char_set)
encoder={k:v for v,k in enumerate(char_set)}
one_hot_encoder=eye(dict_size)
decoder={v:k for v,k in enumerate(char_set)}
encoded_string= [encoder[c] for c in input_string ]
one_hot_encoded_string=[one_hot_encoder[i] for i in encoded_string]

string_width=len(input_string)

output_w=string_width-input_w
x=[];y=[]

for i in range(output_w):
  x.append( one_hot_encoded_string[i:i+input_w]  )
  y.append( one_hot_encoded_string[input_w+i]    )


x=array(x)
y=array(y)
x.shape
x = x.reshape(( output_w, input_w, dict_size))


# 2. Keras Model 
model = Sequential() 
model.add(LSTM(input_w, activation = 'relu', batch_input_shape=(1,input_w,dict_size),stateful=True ) )
model.add(Dense(dict_size) )
model.summary() 
 
# 3. Training 
model.compile(optimizer='adam', loss='mse') 
model.fit(x, y, epochs=100, batch_size=1) 


# 4.Test 

def test_it(test_string,y,debug=True):
  x_input = array([encoder[c] for c in test_string ] ) 
  x_input=[one_hot_encoder[i] for i in x_input]
  x_input=array(x_input)
  x_input = x_input.reshape((1,input_w,dict_size))
  yhat = model.predict(x_input) 
  out=decoder[argmax(yhat)] 
  isOk= y==out
  if debug:
    print(f"# {test_string} →  {out}   {isOk}") 
  return isOk
  
print(f'# InputString : {input_string}')
okCount=0
for s in range(output_w):
  if test_it(input_string[s:input_w+s],input_string[input_w+s:input_w+s+1],False):
    okCount+=1
okPct=okCount/output_w * 100
print(f' {okPct}% : {okCount} / {output_w}')

Model: "sequential_99"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_107 (LSTM)              (1, 4)                    336       
_________________________________________________________________
dense_132 (Dense)            (1, 16)                   80        
Total params: 416
Trainable params: 416
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
46/46 [==============================] - 0s 2ms/step - loss: 0.0651
Epoch 2/100
46/46 [==============================] - 0s 2ms/step - loss: 0.0613
Epoch 3/100
46/46 [==============================] - 0s 2ms/step - loss: 0.0595
Epoch 4/100
46/46 [==============================] - 0s 2ms/step - loss: 0.0580
Epoch 5/100
46/46 [==============================] - 0s 2ms/step - loss: 0.0573
Epoch 6/100
46/46 [==============================] - 0s 2ms/step - loss: 0.0568
Epoch 7/100
46/46 [==================